# Orbit Fitting Tutorial Group Project 4
## Author:  Jason Wang (Northwestern University/CIERA)

Run the [SSW2024_PSF_Setup_Subtraction](https://colab.research.google.com/drive/1AmrmAj459MwUt4kkUbfquxnf1GkyO_F2?usp=share_link) notebook to download the data including the `example_astrometry.csv` as described in the *Create an orbitize data file* section below.  The setup notebook needs to just be run **once** once for Hands-on Session IV.

This tutorial shows how to fit the visual orbit of a directly imaged planet using `orbitize!`.

See the Group Project Idea 4 at the end of [SSW2024_ImagingExoplanets.ipynb](https://colab.research.google.com/drive/1XVUe9-qYAQs_9VVszxuw8Mih49dnhZjX?usp=sharing).

## Google Colab Usage
*Please read (don't just hit run) the information given above each code cell as there are separate install cells for Colab*
&#128992;
*and running Python on your computer*
&#128309;.

**Confirm login account**
* Please make sure to be logged in with the Google account you want to use for the exercises before running the code cells below. You can check by clicking the circular account icon in the top right corner of the colab notebook.

**Working directory**
* Note: The software and data will be installed in a directory called "SSW2024/PSF_Subtraction" in your Google drive. This directory will be created if it does not exist.

**Running cells**
* Run cells individually by clicking on the triangle on each cell

**To Restart runtime**
*   Click on Runtime menu item
*   Select Restart session
*   Select Run code cells individually from the top

**To Recreate runtime**
*   Click on Runtime menu item
*   Select Disconnect and Delete runtime
*   Select Run code cells individually from the top

**To Exit:**
*   Close the browser window

## &#128992; Install `orbitize` in Google Colab
**Run the cell below**


In [ ]:
!pip install orbitize

## &#128992; Setup Google Drive directory

#### &#128992; **Run the next cell to mount the Google Drive**

In [ ]:
# You will be prompted to Permit this notebook to access your Google Drive files - Click on "Connect to Google Drive"
# You will then be prompted to Choose an account - click on your preferred Google account
# You will then confirm that Google Drive for desktop wants to access your Google Account - scroll to click "Continue"
# You may get another prompt to allow additional access for this to work - scroll to click "Continue"

from google.colab import drive
drive.mount('/content/drive')

#### &#128992; **Run the next cell to define the ssw_dir**

In [ ]:
# PSF Subtraction
PSF_dir = 'SSW2024/PSF_Subtraction' #@param {type:"string"}

#### &#128992; **Run the next cell to change to the PSF_Subtraction directory**

In [ ]:
# change to the PSF Subtraction Directory
import os

# Google top level drive dir
drive_dir = "/content/drive/MyDrive/"

# ssw_dir directory path
PSF_path = os.path.join(drive_dir, PSF_dir)

# Change to the pRT_path
os.chdir(PSF_path)

## &#128309; Setup Python running on your own computer directory

**Run the Next cell**

In [ ]:
# Enter the full path to your PSF_Subtraction directory where you downloaded the data in the Setup notebbook

# For example:  '/Users/jsmith/PSF_Subtraction'
import os
absolute_path_to_the_data_folder_on_your_machine = '' # Please complete!
os.chdir(absolute_path_to_the_data_folder_on_your_machine)

# The rest of the notebook cells are for &#128992; Colab or &#128309; running Python on your computer

# Create an orbitize data file.

Follow the example data file [example_astrometry.csv](https://github.com/semaphoreP/ASTRON314-414/blob/main/FinalProject/example_astrometry.csv?plain=1) and make your own data file listing the astrometry you measured from the Imaging Exoplanets Tutorial.

Each row of the data file contains one epoch of astrometry. The fields are as follows:
  1. `epoch`: this is the date in MJD (units of days). For the 2009 dataset, the MJD is 55137. For the 2021 dataset, the MJD is 59404.
  2. `body`: 1 is for the first companion. If you end up fitting more than one companion, the next one is 2, followed by 3, etc... (however, I recommend you fit each planet separately to make the computation easier).
  3. `sep`: the separation of the companion in this epoch in mas
  4. `sep_err`: the error on the separation in mas
  5. `pa`: the position angle of the companion in this epoch in degrees
  6. `pa_err`: the error on the position angle in degrees.

# Run the orbitize fit

In [ ]:
import numpy as np
import orbitize
import orbitize.driver

import matplotlib.pyplot as plt
%matplotlib inline

# Set up the system

Look up what should be used for the total mass and parallax of the system by following values used in the literature. The numbers here are totally wrong!!

In [ ]:
myDriver = orbitize.driver.Driver('example_astrometry.csv'.format(orbitize.DATADIR), # path to data file
                                  'OFTI', # name of algorithm for orbit-fitting
                                  1, # number of secondary bodies in system
                                  1.22, # total mass [M_sun]
                                  56.95) # total parallax of system [mas]


# Find orbits

We will look the find 1000 possible orbital solutions. Note that this could take 1-2 minutes. If you try to fit both planets at the same time, it may take much longer.

In [ ]:
s = myDriver.sampler
orbits = s.run_sampler(1000)

# Plot orbit and range of possible orbital parameters

In [ ]:
my_results = s.results

epochs = my_results.system.data_table['epoch']

orbit_figure = my_results.plot_orbits(
    start_mjd=epochs[0] # Minimum MJD for colorbar (here we choose first data epoch)
)

In [ ]:
sma = np.array([x[s.system.param_idx['sma1']] for x in orbits])
ecc = np.array([x[s.system.param_idx['ecc1']] for x in orbits])
i = np.degrees(np.array([x[s.system.param_idx['inc1']] for x in orbits]))

plt.figure(figsize=(10,3))
plt.subplot(131)
logbins = np.logspace(np.log10(np.min(sma)),np.log10(np.max(sma)),30)
plt.hist(sma, bins=logbins)
plt.gca().set_xscale("log")
plt.xlabel('orbital semi-major axis [AU]')
plt.ylabel('occurrence')

plt.subplot(132)
plt.hist(ecc, bins=30)
plt.xlabel('eccentricity [0,1]')
plt.ylabel('occurrence')

plt.subplot(133)
plt.hist(i, bins=30)
plt.xlabel('inclination angle [deg]')
plt.ylabel('occurrence')

plt.tight_layout()

print("median SMA", np.median(sma))
print("median ecc", np.median(ecc))
print("median Inc", np.median(i))
